In [4]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import shutil
import random

def split_data(src_path):
    class_names = [d for d in os.listdir(src_path) if os.path.isdir(os.path.join(src_path, d))]
    
    if class_names:
        for item in class_names:
            data_dir = os.path.join(src_path, item)  # Đường dẫn đến thư mục dữ liệu gốc
            train_dir = f'data/train/{item}'  # Thư mục để lưu dữ liệu train
            test_dir = f'data/test/{item}'  # Thư mục để lưu dữ liệu test

            # Xóa thư mục nếu đã tồn tại
            shutil.rmtree(train_dir, ignore_errors=True)
            shutil.rmtree(test_dir, ignore_errors=True)

            # Tạo thư mục train và test
            os.makedirs(train_dir, exist_ok=True)
            os.makedirs(test_dir, exist_ok=True)

            # Lấy danh sách tất cả các tệp ảnh .jpg
            files = [file for file in os.listdir(data_dir) if file.endswith('.jpg')]
            random.shuffle(files)

            # Chia dữ liệu vào thư mục train và test
            split_index = int(len(files) * 0.8)  # 80% cho train
            train_files = files[:split_index]
            test_files = files[split_index:]

            for file in train_files:
                shutil.copy(os.path.join(data_dir, file), train_dir)
            for file in test_files:
                shutil.copy(os.path.join(data_dir, file), test_dir)

        print("Dữ liệu đã được chia thành công vào thư mục train và test.")
    else:
        print("Dữ liệu tệp nguồn trống không thể chia.")
    
    return bool(class_names)


def create_cnn_model(input_shape, num_classes):
    model = Sequential()

    # Lớp tích chập đầu tiên với số bộ lọc giảm xuống
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))  # Dropout để giảm overfitting

    # Lớp tích chập thứ hai
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # Lớp tích chập thứ ba
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # Flatten để chuyển sang lớp fully connected
    model.add(Flatten())
    
    # Lớp fully connected
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Lớp đầu ra
    model.add(Dense(num_classes, activation='softmax'))

    # Compile mô hình với Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model
def train_model(train_data_dir, test_data_dir, input_shape=(150, 150, 3), epochs=50):
    if not split_data('data/src'):
        return

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.2,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    model = create_cnn_model(input_shape=input_shape, num_classes=train_generator.num_classes)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=test_generator.samples // test_generator.batch_size,
        # callbacks=[reduce_lr, early_stopping]
    )

    loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
    print(f'Test Accuracy: {accuracy:.2f}')

    model.save('cnn_model_improved.keras')

# Cách sử dụng
train_data_directory = 'data/train'
test_data_directory = 'data/test'
train_model(train_data_directory, test_data_directory)


Dữ liệu đã được chia thành công vào thư mục train và test.
Found 762 images belonging to 5 classes.
Found 192 images belonging to 5 classes.


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 528ms/step - accuracy: 0.2571 - loss: 1.6686

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 16s 625ms/step - accuracy: 0.2578 - loss: 1.6662 - val_accuracy: 0.3490 - val_loss: 1.5602
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2812 - loss: 1.7150  
Epoch 3/50


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 410ms/step - accuracy: 0.3087 - loss: 1.5191 - val_accuracy: 0.4583 - val_loss: 1.5401
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.4062 - loss: 1.5105
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 406ms/step - accuracy: 0.3327 - loss: 1.5501 - val_accuracy: 0.6406 - val_loss: 1.5108
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.5312 - loss: 1.3267
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 453ms/step - accuracy: 0.3803 - loss: 1.4089 - val_accuracy: 0.6771 - val_loss: 1.4723
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - accuracy: 0.3438 - loss: 1.4133
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 446ms/step - accuracy: 0.4422 - loss: 1.3711 - val_accuracy: 0.7708 - val_loss: 1.3915
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.4062 - loss: 1.4084   
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 466ms/step - accuracy: 0.4656 - loss: 1.3404 - val_accuracy: 0.9479 - val_loss: 1.3496
Epoch 12/50
23/23 ━━━━

In [5]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
import shutil
import random
from flask_socketio import SocketIO

# Khởi tạo socketio
socketio = SocketIO()

def split_data(src_path):
    class_names = [d for d in os.listdir(src_path) if os.path.isdir(os.path.join(src_path, d))]
    
    if class_names:
        for item in class_names:
            data_dir = os.path.join(src_path, item)  # Đường dẫn đến thư mục dữ liệu gốc
            train_dir = f'data/train/{item}'  # Thư mục để lưu dữ liệu train
            test_dir = f'data/test/{item}'  # Thư mục để lưu dữ liệu test

            # Xóa thư mục nếu đã tồn tại
            shutil.rmtree(train_dir, ignore_errors=True)
            shutil.rmtree(test_dir, ignore_errors=True)

            # Tạo thư mục train và test
            os.makedirs(train_dir, exist_ok=True)
            os.makedirs(test_dir, exist_ok=True)

            # Lấy danh sách tất cả các tệp ảnh .jpg
            files = [file for file in os.listdir(data_dir) if file.endswith('.jpg')]
            random.shuffle(files)

            # Chia dữ liệu vào thư mục train và test
            split_index = int(len(files) * 0.8)  # 80% cho train
            train_files = files[:split_index]
            test_files = files[split_index:]

            for file in train_files:
                shutil.copy(os.path.join(data_dir, file), train_dir)
            for file in test_files:
                shutil.copy(os.path.join(data_dir, file), test_dir)

        print("Dữ liệu đã được chia thành công vào thư mục train và test.")
    else:
        print("Dữ liệu tệp nguồn trống không thể chia.")
    
    return bool(class_names)

# Callback để phát tiến độ qua Socket.IO
class ProgressCallback(Callback):
    def __init__(self, total_epochs):
        super().__init__()
        self.total_epochs = total_epochs

    def on_epoch_end(self, epoch, logs=None):
        # Tính toán tiến độ
        progress = int((epoch + 1) * 100 / self.total_epochs)
        socketio.emit('progress', {'percent': progress})  # Phát tiến độ sau mỗi epoch
        print(f'Epoch {epoch+1}/{self.total_epochs} - Progress: {progress}%')

def create_cnn_model(input_shape, num_classes):
    model = Sequential()

    # Lớp tích chập đầu tiên với số bộ lọc giảm xuống
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))  # Dropout để giảm overfitting

    # Lớp tích chập thứ hai
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # Lớp tích chập thứ ba
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # Flatten để chuyển sang lớp fully connected
    model.add(Flatten())
    
    # Lớp fully connected
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Lớp đầu ra
    model.add(Dense(num_classes, activation='softmax'))

    # Compile mô hình với Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def train_model(train_data_dir, test_data_dir, input_shape=(150, 150, 3), epochs=50, socketio=None):
    if not split_data('data/src'):
        return

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.2,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical'
    )

    # Sử dụng mô hình CNN được định nghĩa trước
    model = create_cnn_model(input_shape=input_shape, num_classes=train_generator.num_classes)

    # Khởi tạo các callback cho giảm learning rate và phát tiến độ
    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
    # early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    progress_callback = ProgressCallback(total_epochs=epochs)  # Callback để gửi tiến độ

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}:')
        # Giả lập tiến độ từng epoch
        model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // train_generator.batch_size,
            epochs=1,
            validation_data=test_generator,
            validation_steps=test_generator.samples // test_generator.batch_size,
        )
        
        # Tính tiến độ và gửi về client
        progress = (epoch + 1) * 100 / epochs
        if socketio:  # Kiểm tra socketio có tồn tại không
            socketio.emit('progress', {'percent': progress})

    # Phát tín hiệu hoàn thành
    if socketio:
        socketio.emit('progress', {'percent': 100})

    loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
    print(f'Test Accuracy: {accuracy:.2f}')

    # Lưu mô hình
    model.save('cnn_model_improved2.keras')
    socketio.emit('progress', {'percent': 100})

train_model('data/train', 'data/test', socketio=socketio)

 5/23 ━━━━━━━━━━━━━━━━━━━━ 6s 372ms/step - accuracy: 0.3490 - loss: 1.4694

KeyboardInterrupt: 

In [2]:
import numpy as np
import os, uuid
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model('cnn_model_improved.keras')
# Dự đoán với mô hình
def predict_image(img_path):
    """Hàm dự đoán hình ảnh."""
    img = image.load_img(img_path, target_size=(150, 150))  # Thay đổi kích thước hình ảnh
    img_array = image.img_to_array(img) / 255.0  # Chia pixel cho 255
    img_array = np.expand_dims(img_array, axis=0)  # Thêm chiều cho batch

    # Dự đoán
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class

# Gọi hàm dự đoán với một hình ảnh cụ thể
rs = predict_image('data/user_1_4.jpg')
data_path = 'data/test'
class_names = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
print(f'result: {class_names[int(rs)]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
result: user_1


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8488\729829704.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f'result: {class_names[int(rs)]}')


In [1]:
from models import User
from database import get_session
from datetime import datetime

session = get_session()

users = [
    User(
        name="Bông",
        email="alicesmith@example.com",
        created_at=datetime.utcnow(),
        updated_at=datetime.utcnow(),
        face_encoding="data/src/user_1"
    ),
    User(
        name="Gấu",
        email="bobjohnson@example.com",
        created_at=datetime.utcnow(),
        updated_at=datetime.utcnow(),
        face_encoding="data/src/user_2"
    ),
    User(
        name="Sạc",
        email="bobjohnson@example.com",
        created_at=datetime.utcnow(),
        updated_at=datetime.utcnow(),
        face_encoding="data/src/user_3"
    ),
    User(
        name="Bà",
        email="bongba@example.com",
        created_at=datetime.utcnow(),
        updated_at=datetime.utcnow(),
        face_encoding="data/src/user_4"
    ),
    User(
        name="Mũ",
        email="muba@example.com",
        created_at=datetime.utcnow(),
        updated_at=datetime.utcnow(),
        face_encoding="data/src/user_5"
    )
]

# Thêm tất cả vào session và lưu vào cơ sở dữ liệu
session.add_all(users)
session.commit()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10976\633188199.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  created_at=datetime.utcnow(),
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10976\633188199.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  updated_at=datetime.utcnow(),
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10976\633188199.py:18: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  created_at=datetime.utcnow(),
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10976\633188199.py:19: DeprecationWarning: dat

In [15]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from database import get_session



# Khởi tạo session
session = get_session()

# Lấy tất cả các bản ghi từ bảng users
users = session.query(User).all()

# Lặp qua từng bản ghi và thay thế \ bằng /
for user in users:
    if user.face_encoding:  # Kiểm tra nếu face_encoding không phải là None
        original_face_encoding = user.face_encoding
        updated_face_encoding = original_face_encoding.replace('\\', '/')
        user.face_encoding = updated_face_encoding
        print(f"Cập nhật face_encoding từ '{original_face_encoding}' thành '{updated_face_encoding}'.")

# Lưu lại các thay đổi vào cơ sở dữ liệu
session.commit()

# Đóng session
session.close()


Cập nhật face_encoding từ 'data/src/user_1' thành 'data/src/user_1'.
Cập nhật face_encoding từ 'data/src/user_2' thành 'data/src/user_2'.
Cập nhật face_encoding từ 'data/src/user_3' thành 'data/src/user_3'.
Cập nhật face_encoding từ 'data\src\user_4' thành 'data/src/user_4'.
Cập nhật face_encoding từ 'data\src\user_5' thành 'data/src/user_5'.


StatementError: (builtins.AttributeError) module 'datetime' has no attribute 'utcnow'
[SQL: UPDATE users SET updated_at=?, face_encoding=? WHERE users.id = ?]
[parameters: [{'face_encoding': 'data/src/user_4', 'users_id': 4}, {'face_encoding': 'data/src/user_5', 'users_id': 5}]]

In [4]:
import sqlite3

# Kết nối đến cơ sở dữ liệu
conn = sqlite3.connect('users.db')
cursor = conn.cursor()


cursor.execute("DELETE FROM face_records")
conn.commit()
# cursor.execute("DELETE FROM users")
# conn.commit()

print(f"Số bản ghi đã bị xóa: {cursor.rowcount}")

conn.close()


Dữ liệu đã được chia thành công vào thư mục train và test.


True